# Frequency-wavenumber tutorial for Virtual ASA 2024

## Author:  Kathleen E. Wage, George Mason University 
## Contact:  k.e.wage@ieee.org

### Abstract 
This notebook demonstrates how to compute the frequency-wavenumber spectrum given time series data recorded on a uniform vertical line array.  The slides below briefly review basic array processing concepts.  Following the conceptual overview, the tutorial provides code to read in a data file, compute the average frequency-wavenumber spectrum, and plot the spectrum.  The code allows the user to specify block lengths, FFT sizes, and windows/tapers. The slides at the end provide some suggestions for further reading.

Several data files are provided along with this tutorial:
* asa_synth1.npz containing 5 test signals
    * CW signal: frequency 20 Hz, $\theta=87$ deg
    * CW signal: frequency 20 Hz, $\theta=93$ deg
    * CW signal: frequency 20 Hz, $\theta=135$ deg
    * CW signal: frequency 40 Hz, $\theta=90$ deg
    * CW signal: frequency 41 Hz, $\theta=90$ deg
* asa_synth2.npz
    * CW signal: frequency 20 Hz, $\theta=45$ deg
    * Broadband noise: $\theta=180$ deg
    * LFM signal: sweep 10 to 40 Hz, $\theta=110$ deg
* asa_synth3.npz containing 6 test signals     
    **Challenge Problem:** Can you find the signals in the asa_synth3 data set?
* spib_350hz.npz
    * 350 Hz SACLANT sonar data from the [Signal Processing Information Base website](http://spib.linse.ufsc.br/sonar.html) 

All of the synthetic data files were generated assuming simple planewave propagation for an environment with a sound speed of 1500 m/s.  


![title](outcome.png)

![title](clean_planewaves.png)

![title](noisy_planewaves.png)

![title](spectrum_compute1.png)

![title](spectrum_compute2.png)

![title](windows.png)

![title](komega1.png)

![title](komega2.png)

![title](komega3.png)

![title](asa_synth1_1.png)
![title](asa_synth1_2.png)
![title](asa_synth1_3.png)
![title](asa_synth1_4.png)

![title](asa_synth2.png)
![title](sampling_space1.png)
![title](sampling_space2.png)



In [ ]:
# Import standard libraries for plotting and scientific computation
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal 
from scipy.fft import fft, fftfreq, fftshift

# Ignore errors in computing np.log10 for log-scale plots
np.seterr(divide = 'ignore') 

# Set seaborn plotting variables
sns.set_context("notebook")
sns.set_palette("colorblind")

# A few of my own plot parameters
plt.rcParams['figure.figsize']=(11.5,6)
 
# Set up logical variable to decide whether the save a plot
saveplots=True


In [ ]:
# This cell reads the data and sets up the parameters for the frequency-wavenumber plot

# Read data from .npz file
datfile='asa_synth1.npz'
#datfile='spib_350hz.npz'
npzdata=np.load(datfile)
fs=npzdata['fs']
dz=npzdata['dz']
x=npzdata['x']

# Determine number of sensors and time samples in data from the array size
[nrcv,nsig]=x.shape

# Assumed sound speed:  used to mark the visible region on the plot
c=1500

# User sets up these processing parameters
# Parameters:  block length, overlap (or skip), window types, size of FFTs
twintype='hann'   # Temporal window type
swintype='boxcar'   # Spatial window type
blklength=1024
nskip=np.rint(blklength/2).astype(int)   # Set for 50% overlap
Ntempfft=4096
Nspatfft=1024

In [ ]:
# This is the code cell that computes the average frequency-wavenumber spectra

# Generate the temporal and spatial windows
# Note that Twinmtx and Swinmtx are matrices containing copies of the window 
# This facilitates tapering the block of data with a matrix multiply

twin=signal.get_window(twintype,blklength)
twin=twin/(np.vdot(twin,twin))           # normalization
Twinmtx=np.outer(np.ones((nrcv,1)),twin)

swin=signal.get_window(swintype,nrcv)
swin=swin/(np.vdot(swin,swin))            # normalization
Swinmtx=np.outer(swin,np.ones((1,Ntempfft)))

# Compute starting indices of the time blocks and the number of blocks
nstart=range(0,nsig-blklength,nskip)
nblk=len(nstart)
print('Number of blocks: %i' % (nblk))

avgkomega=np.zeros((Nspatfft,Ntempfft))
fvec=fftshift(fftfreq(Ntempfft,1/fs))
kzvec=fftshift(fftfreq(Nspatfft,dz))*2*np.pi

for ind in nstart:
    datablk=x[:,ind:ind+blklength:1]
    tempfft=fftshift(fft(datablk*Twinmtx,Ntempfft,1),1)
    spacefft=fftshift(fft(tempfft*Swinmtx,Nspatfft,0),0)
    komega=np.square(np.abs(spacefft))
    avgkomega=avgkomega+komega/nblk

In [ ]:
# Plot the resulting frequency-wavenumber spectrum

maxval=np.max(avgkomega) # Use to normalize so peak of spectrum is at 0 dB

# set up lines to mark visible region
vis1=2*np.pi*fvec/c
vis2=-2*np.pi*fvec/c

fig,ax=plt.subplots()

ax.plot(fvec[0,:],vis1[0,:],'w--')
ax.plot(fvec[0,:],vis2[0,:],'w-.')
im=ax.imshow(10*np.log10(avgkomega/maxval), origin='upper', aspect='auto', cmap='viridis', 
             extent=[fvec[0,0],fvec[0,-1],kzvec[0,-1],kzvec[0,0]],vmin=-50,vmax=0)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Wavenumber $k_z$')
ax.legend(['$\\theta=0$ deg', '$\\theta=180$ deg'],loc='lower right')
ax.set_xlim(0,fvec[0,-1])          # Set to show only positive frequencies
tstr='File: %s, Block length: %i pts, Time window: %s, Space window: %s' % (datfile,blklength,twintype,swintype)
ax.set_title(tstr)

# Add colorbar and layout adjustments
fig.colorbar(im, label='Spectrum (dB)')
fig.tight_layout()

plt.show()


In [ ]:
# Save the plot to an .eps file if desired

if saveplots:
    figfile='figs/%s_block%i_twin_%s_swin_%s.eps' % (datfile, blklength,
                                                     twintype, swintype)
    fig.savefig(figfile,bbox_inches="tight")

![title](techreport.png)

![title](ack.png)
